In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

In [2]:
df = pd.read_csv('Collection.csv', names=['Service', 'ConNo', 'Charge', 'in', 'out', 'id', 'LineOp', 'totality', 'code','type', 'length', 'stop'])
df.head()

,Service,ConNo,Charge,in,out,id,LineOp,totality,code,type,length,stop
0,Import Demurrage charges,AMFU8455669,328,2016-02-02 13:29:50.850,2015-12-17 00:00:00.000,1,MSC,FULL,42G0,DC,40,NaN
1,Import Demurrage charges,AMFU8615670,25,2016-01-04 12:58:37.553,2015-12-21 00:00:00.000,2,MSC,FULL,42G0,DC,40,NaN
2,Import Demurrage charges,BMOU5639927,25,2016-01-04 12:58:41.373,2015-12-21 00:00:00.000,3,MSC,FULL,42G0,DC,40,NaN
3,Import Demurrage charges,CAIU2826800,6,2016-01-04 16:11:02.557,2015-12-26 00:00:00.000,4,MSC,FULL,22G0,DC,20,NaN
4,Import Demurrage charges,CAIU8841314,15,2016-01-07 12:30:14.927,2015-12-26 00:00:00.000,5,MSC,FULL,42G0,DC,40,NaN


In [3]:
i=0
for ex in df['out']:
    if type(ex) == float:
        df.drop(i, axis=0, inplace=True)
    i=i+1

In [4]:
i=0
for (din, dout) in zip(df['in'], df['out']):
    ids = datetime.strptime(din, '%Y-%m-%d %H:%M:%S.%f')
    ods = nds = datetime.strptime(dout, '%Y-%m-%d %H:%M:%S.%f')
    if (ids-ods) < timedelta(seconds=0):
        df.drop(i, axis=0, inplace=True)
    i=i+1

In [5]:
td = []
for (din, dout) in zip(df['in'], df['out']):
    ids = datetime.strptime(din, '%Y-%m-%d %H:%M:%S.%f')
    ods = nds = datetime.strptime(dout, '%Y-%m-%d %H:%M:%S.%f')
    dif=ids-ods
    dif = dif.days
    td.append(dif)
df = df.assign(td=td)

In [ ]:
sns.pairplot(df)

In [ ]:
df.drop(['Service', 'ConNo', 'code', 'in', 'out', 'id', 'totality', 'stop'], 1 , inplace=True)

In [ ]:
# Get list of categorical variables
s = (df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])

In [ ]:
fig = px.scatter(df, x="td", y="Charge", color = 'type')
fig.show()

In [ ]:
x = df.drop("Charge",axis=1)
y = df["Charge"]

xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
x.isnull().value_counts()

In [ ]:
lm = LinearRegression().fit(xTrain,yTrain)
pls = PLSRegression().fit(xTrain,yTrain)
ridge = Ridge().fit(xTrain,yTrain)
lasso = Lasso().fit(xTrain,yTrain)
elasticnet = ElasticNet().fit(xTrain,yTrain)
knnr = KNeighborsRegressor().fit(xTrain,yTrain)
cartr = DecisionTreeRegressor(random_state=42).fit(xTrain,yTrain)
baggr = BaggingRegressor(random_state=42,bootstrap_features=True,verbose=False).fit(xTrain,yTrain)
rfr = RandomForestRegressor(random_state=42,verbose=False).fit(xTrain,yTrain)
gbmr = GradientBoostingRegressor(verbose=False).fit(xTrain,yTrain)
xgbr = XGBRegressor().fit(xTrain,yTrain)
lgbmr = LGBMRegressor().fit(xTrain,yTrain)
catbr = CatBoostRegressor(verbose=False).fit(xTrain,yTrain)